In [ ]:
import os
from PIL import Image
import numpy as np                # funciones numéricas (arrays, matrices, etc.)
import matplotlib.pyplot as plt   # funciones para representación gráfica
#from skimage import io
 
carpeta = "../1Dataset/"
cont = 0
width = 28
height = 28
for archivo in os.listdir(carpeta):
    cont = cont + 1
I = np.zeros([cont, width, height])
cont = 0
for archivo in os.listdir(carpeta):
    #print(os.path.join(carpeta,archivo))
    foto = Image.open(os.path.join(carpeta,archivo))
    datos = foto.getdata()
 
    #para el calculo del promedio se utilizara la division entera con el operador de division doble "//" para evitar decimales

    promedio = [(datos[x][0] + datos[x][1] + datos[x][2]) // 3 for x in range(len(datos))]
    imagen_gris = Image.new('L', foto.size)
    imagen_gris.putdata(promedio)
    imagen_gris2 = imagen_gris.resize((width, height), Image.NEAREST) 
    I[cont, :, :] = imagen_gris2
    #I[cont, :, :] = io.imread(os.path.join(carpeta,archivo))
    cont = cont + 1
    #plt.imshow(I)
    

carpeta = "../2Dataset/"
cont2 = 0
for archivo in os.listdir(carpeta):
    cont2 = cont2 + 1
I2 = np.zeros([cont2, width, height])
cont2 = 0
for archivo in os.listdir(carpeta):
    #print(os.path.join(carpeta,archivo))
    
    if np.random.random() < 0.12:
        try: 
            foto2 = Image.open(os.path.join(carpeta,archivo))
            datos2 = foto2.getdata()
            #para el calculo del promedio se utilizara la division entera con el operador de division doble "//" para evitar decimales
            promedio2 = [(datos2[x][0] + datos2[x][1] + datos2[x][2]) // 3 for x in range(len(datos2))]
            imagen_gris3 = Image.new('L', foto2.size)
            imagen_gris3.putdata(promedio2)
            imagen_gris4 = imagen_gris3.resize((width, height), Image.NEAREST) 
            I2[cont2, :, :] = imagen_gris4
            #I[cont, :, :] = io.imread(os.path.join(carpeta,archivo))
            cont2 = cont2 + 1
            #plt.imshow(I)
        except:
            print("Imagen Blanco y negro, " + archivo)
        

In [ ]:
cont2

In [ ]:
I3 = np.zeros([cont, width * height])
for i in range(0, cont):
        I3[i, :] = I[i, :, :].ravel()
I4 = np.zeros([cont2, width * height])
for i in range(0, cont2):
        I4[i, :] = I2[i, :, :].ravel()

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.ensemble import IsolationForest
from sklearn import svm
from sklearn.externals import joblib

# Apply standard scaler to output from resnet50
X_train = I3[:10000, :]
X_test = I3[10001:, :]

ss = StandardScaler()
ss.fit(X_train)
X_train = ss.transform(X_train)
X_test = ss.transform(X_test)

X_test2 = ss.transform(I4)
# Take PCA to reduce feature space dimensionality
pca = PCA(n_components=400, whiten=True)
pca = pca.fit(X_train)
print('Explained variance percentage = %0.2f' % sum(pca.explained_variance_ratio_))
X_train = pca.transform(X_train)
X_test = pca.transform(X_test)
X_test2 = pca.transform(X_test2)
# Train classifier and obtain predictions for OC-SVM
oc_svm_clf = svm.OneClassSVM(gamma='auto', kernel='rbf', nu=0.27)  # Obtained using grid search

if_clf = IsolationForest(bootstrap =True, n_jobs = -1, contamination=0.27, max_features=1.0, max_samples=1.0, n_estimators=400)  # Obtained using grid search

oc_svm_clf.fit(X_train)

if_clf.fit(X_train)

oc_svm_preds = oc_svm_clf.predict(X_test)

if_preds = if_clf.predict(X_test)

oc_svm_preds2 = oc_svm_clf.predict(X_test2)

if_preds2 = if_clf.predict(X_test2)

Accuracy = 0
matrix_C = np.zeros([2, 2])
for X2 in range(0, if_preds.shape[0]):
    if if_preds[X2]==1: #Verdaderos Positivos
        Accuracy = Accuracy + 1
        matrix_C[0, 0] = matrix_C[0, 0] + 1 
    else:
        matrix_C[0, 1] = matrix_C[0, 1] + 1 
    
for X2 in range(0, if_preds2.shape[0]):
    if if_preds2[X2]==-1: #Verdaderos Negativos
        Accuracy = Accuracy + 1
        matrix_C[1, 1] = matrix_C[1, 1] + 1 
    else:
        matrix_C[1, 0] = matrix_C[1, 0] + 1 
Accuracy = Accuracy / (if_preds.shape[0]+if_preds2.shape[0]); 
print("Accurracy Isolation Forest: " + str(Accuracy))

Accuracy2 = 0
matrix_C2 = np.zeros([2, 2])
for X2 in range(0, oc_svm_preds.shape[0]):
    if oc_svm_preds[X2]==1: #Verdaderos Positivos
        Accuracy2 = Accuracy2 + 1
        matrix_C2[0, 0] = matrix_C2[0, 0] + 1 
    else:
        matrix_C2[0, 1] = matrix_C2[0, 1] + 1 
for X2 in range(0, oc_svm_preds2.shape[0]):
    if oc_svm_preds2[X2]==-1: #Verdaderos Negativos
        Accuracy2 = Accuracy2 + 1
        matrix_C2[1, 1] = matrix_C2[1, 1] + 1 
    else:
        matrix_C2[1, 0] = matrix_C2[1, 0] + 1 
Accuracy2 = Accuracy2 / (oc_svm_preds.shape[0] + oc_svm_preds2.shape[0])
print("Accurracy SVM " + str(Accuracy2))

vFileName = 'Clasificador_de_caras.joblib.pkl'
vFilePathRelative = 'Save Models/' + vFileName
vDirPath = os.path.dirname(os.path.realpath(vFilePathRelative))
vFilePath = os.path.join(vDirPath, vFileName)
if Accuracy > Accuracy2:
    vDump = joblib.dump(if_clf, vFilePathRelative, compress = 9)
else: 
    vDump = joblib.dump(oc_svm_clf, vFilePathRelative, compress = 9)
print("Model Saved")

In [ ]:
import sys
import numpy
numpy.set_printoptions(threshold=sys.maxsize)

In [ ]:
if_preds

In [ ]:
matrix_C2

In [ ]:
matrix_C

In [ ]:
pca.explained_variance_[:80]

In [ ]:
from sklearn.externals import joblib
import pickle

fid = open('Save Models/Clasificador_de_caras.joblib.pkl', 'rb')
joblib_Model = joblib.load(fid)
PrediccionTest = joblib_Model.predict(X_test)

In [ ]:
PrediccionTest